In [ ]:
import pandas as pd
import numpy as np 
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
!pip install ndjson


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1BNCRmYK0x6qx6knpGrZPGqPuQiEh7NPN
!gdown https://drive.google.com/uc?id=1BTIae-YYyTvZNyor3Od5Nkms4Y1mH_0O
!gdown https://drive.google.com/uc?id=1B4jUdGkkCwYVGuj894EQHDQxWNJj78g0
!gdown https://drive.google.com/uc?id=1B20mc0udeAymHxd-0K5vOkOBmOPp1lt4
!gdown https://drive.google.com/uc?id=1B9ptrAc_ayFe2xw1I2VtGajBFCfBiEyB
!gdown https://drive.google.com/uc?id=1BJYnALyY0ndcKooYFf1CJkCro_lbEtJZ

#!gdown https://drive.google.com/uc?id=1BTIae-YYyTvZNyor3Od5Nkms4Y1mH_0O
#!gdown https://drive.google.com/uc?id=1BTIae-YYyTvZNyor3Od5Nkms4Y1mH_0O


Downloading...
From: https://drive.google.com/uc?id=1BNCRmYK0x6qx6knpGrZPGqPuQiEh7NPN
To: /content/train_tallyqa.json
100% 47.2M/47.2M [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BTIae-YYyTvZNyor3Od5Nkms4Y1mH_0O
To: /content/test_tallqa.json
100% 7.51M/7.51M [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B4jUdGkkCwYVGuj894EQHDQxWNJj78g0
To: /content/instances_train2014.json
100% 333M/333M [00:04<00:00, 80.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B20mc0udeAymHxd-0K5vOkOBmOPp1lt4
To: /content/captions_train2014.json
100% 66.8M/66.8M [00:00<00:00, 231MB/s]
Downloading...
From: https://drive.google.com/uc?id=1B9ptrAc_ayFe2xw1I2VtGajBFCfBiEyB
To: /content/instances_val2014.json
100% 161M/161M [00:01<00:00, 120MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BJYnALyY0ndcKooYFf1CJkCro_lbEtJZ
To: /content/captions_val2014.json
100% 32.4M/32.4M [00:00<00:00, 134MB/s]


In [ ]:
data_tallqa_train = pd.read_json('/content/train_tallyqa.json')
data_tallqa_test = pd.read_json('/content/test_tallqa.json')
tally_qa = pd.concat([data_tallqa_train,data_tallqa_test], axis=0)
tally_qa['image_origin']= tally_qa['image'].apply(lambda x: x.split('_')[0])
tally_qa = tally_qa[tally_qa['image_origin'].isin(['train2014/COCO', 'val2014/COCO'])].reset_index(drop=True)
tally_qa = tally_qa.drop_duplicates(subset=['image'],keep='first')

In [ ]:
tally_qa.head(2)

,image,answer,data_source,question,image_id,question_id,issimple,image_origin
0,train2014/COCO_train2014_000000247712.jpg,2,imported_vqa,How many skateboards in the photo?,247712,20000000,NaN,train2014/COCO
1,train2014/COCO_train2014_000000464605.jpg,1,imported_vqa,How many skiers are in the air?,464605,20000001,NaN,train2014/COCO


In [ ]:
annFile_instance='/content/instances_train2014.json'
annFile_instance_valid = '/content/instances_val2014.json'
annFile_caption='/content/captions_train2014.json'
annFile_caption_valid = '/content/captions_val2014.json'

In [ ]:
# get all images containing given categories, select one at random
def get_image_ids(path):
    coco =COCO(path)
    animal_data = []
    animals = ['giraffes', 'elephants','zebras','bears','cows','cats','sheep','broccolis','dogs','giraffe','horses','mouse','dog']
    for animal in animals:
      catIds = coco.getCatIds(catNms=animal)
      imgIds = coco.getImgIds(catIds=catIds )
      df = pd.DataFrame(imgIds)
      df['animal']=animal
      animal_data.append(df)
    
    animal_df = pd.concat([df for df in animal_data],ignore_index=True)
    animal_df.columns = ['image_id_coco','animal']
    return animal_df

In [ ]:
animal_df = get_image_ids(annFile_instance)
animal_df_valid = get_image_ids(annFile_instance_valid)
animal_ids = pd.concat([animal_df,animal_df_valid], axis=1)

loading annotations into memory...
Done (t=21.32s)
creating index...
index created!
loading annotations into memory...
Done (t=5.26s)
creating index...
index created!


In [ ]:
animal_df['animal'].value_counts()

dogs         3041
dog          3041
cats         2818
horses       2068
giraffes     1798
giraffe      1798
elephants    1518
cows         1389
broccolis    1340
zebras       1324
mouse        1290
sheep        1105
bears         668
Name: animal, dtype: int64

In [ ]:
animal_merged = animal_df.merge(tally_qa, left_on='image_id_coco', right_on='image_id', how='inner')
animal_merged = animal_merged.drop_duplicates(subset=['image_id'])

In [ ]:
animal_merged['image_id'].nunique()

14462

In [ ]:
animal_merged.shape

(14462, 10)

In [ ]:
import ndjson
f = open(annFile_caption)
s = ndjson.load(f)
data = s[0]

In [ ]:
data.keys()

dict_keys(['info', 'images', 'licenses', 'annotations'])

In [ ]:
data['annotations']
caption_dict = {}
for dic in data['annotations']:
  if(dic['image_id'] not in caption_dict.keys()):
    caption_dict[dic['image_id']] = [dic['caption']]
  else:
    caption_dict[dic['image_id']].append(dic['caption'])
#caption_dict

In [ ]:
annotations = []
a2 = []
a3 = []
a4 = []
a5 = []
for imageid in animal_merged.loc[:,'image_id']:
  annotations.append(caption_dict[imageid][0])
  a2.append(caption_dict[imageid][1])
  a3.append(caption_dict[imageid][2])
  a4.append(caption_dict[imageid][3])
  a5.append(caption_dict[imageid][4])


animal_merged.loc[:,'annot1'] = annotations
animal_merged.loc[:,'annot2'] = a2
animal_merged.loc[:,'annot3'] = a3
animal_merged.loc[:,'annot4'] = a4
animal_merged.loc[:,'annot5'] = a5

In [ ]:
animal_merged.head()

,image_id_coco,animal,image,answer,data_source,question,image_id,question_id,issimple,image_origin,annot1,annot2,annot3,annot4,annot5
0,507908,giraffes,train2014/COCO_train2014_000000507908.jpg,3,imported_vqa,How many animals are there?,507908,20017069,NaN,train2014/COCO,Several giraffes standing around in an animal ...,three giraffes and they are in their pen,Several giraffes look over a fence in their ha...,"The giraffes are different heights, but all ha...",Three giraffe look out from their enclosure.
2,327695,giraffes,train2014/COCO_train2014_000000327695.jpg,2,imported_vqa,How many species of animals do you see?,327695,20012298,NaN,train2014/COCO,some giraffes green grass trees and bushes and...,Giraffes are roaming freely while the people w...,Giraffes walking through a large field of grass.,some kids are watching as giraffes walk around...,"Several giraffes, zebra and sheep with a child..."
5,122899,giraffes,train2014/COCO_train2014_000000122899.jpg,2,tdiuc_templates,How many people are there?,122899,10604586,NaN,train2014/COCO,A woman holding a child next to a wooden fence.,Woman holding a small child at a zoo watching ...,A woman holding a child in her arm behind a fe...,A woman holding a child looking at a giraffe,The giraffe is walking toward the edge of the ...
7,327702,giraffes,train2014/COCO_train2014_000000327702.jpg,2,imported_vqa,How many giraffes are in the picture?,327702,20012724,NaN,train2014/COCO,Woman and infant son hand feeding giraffe at zoo,a woman holding a child is feeding a giraffe,a lady holding a young boy at a zoo feeding a ...,WOMAN FEEDING A GIRAFFE WHILE HOLDING A CHILD,The woman is showing the child how to feed the...
9,25,giraffes,train2014/COCO_train2014_000000000025.jpg,2,imported_vqa,How many giraffes are there?,25,20032316,NaN,train2014/COCO,A giraffe eating food from the top of the tree.,A giraffe standing up nearby a tree,A giraffe mother with its baby in the forest.,Two giraffes standing in a tree filled area.,A giraffe standing next to a forest filled wit...


In [ ]:
animal_merged.shape

(14462, 15)

In [ ]:
animal_merged['image_id'].nunique()

14462

In [ ]:
animal_merged.isnull().sum()

image_id_coco        0
animal               0
image                0
answer               0
data_source          0
question             0
image_id             0
question_id          0
issimple         14462
image_origin         0
annot1               0
annot2               0
annot3               0
annot4               0
annot5               0
dtype: int64

In [ ]:
animal_merged['animal'].value_counts()

dogs         2104
cats         2092
horses       1600
giraffes     1575
elephants    1322
broccolis    1209
cows         1171
zebras       1069
mouse         964
sheep         892
bears         464
Name: animal, dtype: int64

In [ ]:
animal_merged.to_csv('animal_curated_dataset_test.csv')